In [3]:
import pandas as pd
import hashlib


def anonymize_by_sha256(data):
    return hashlib.sha256(data.encode()).hexdigest()


df_hungarian_events = pd.read_csv("Data/athlete_events.csv")
df_hungarian_events = df_hungarian_events.loc[df_hungarian_events["NOC"] == "HUN"]
df_hungarian_events["Name"] = df_hungarian_events["Name"].apply(anonymize_by_sha256)

#### Anonymizing by using SHA-256

In [4]:
df_hungarian_events.head()

208    3d783a420a34bbe55508bbd5f6eaf42e2a5c8804a13ac6...
209    ed7fb6924e1b785bfd0d7d8f53af169ad6488abb1505b7...
210    ed7fb6924e1b785bfd0d7d8f53af169ad6488abb1505b7...
884    557f18d348e2fb24e10d9cc87fb47162eb705d53ae01fe...
885    557f18d348e2fb24e10d9cc87fb47162eb705d53ae01fe...
Name: Name, dtype: object